<a href="https://colab.research.google.com/github/oaarnikoivu/dissertation/blob/master/BERT_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 450kB 2.9MB/s 
     |████████████████████████████████| 1.0MB 14.0MB/s 
     |████████████████████████████████| 870kB 20.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=18d8a3821e9e9bac59bacf5e28c501570e11de3e76633a743fce1135c3307383
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from pathlib import Path

import torch
import transformers as ppb 
import warnings 

warnings.filterwarnings('ignore')

# Load data

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = '/content/drive/My Drive/'

DATA_PATH = Path(path + '/datasets/SemEval/')

train_df = pd.read_csv(DATA_PATH/'train.csv', delimiter=',')
test_df = pd.read_csv(DATA_PATH/'test.csv', delimiter=',')
val_df = pd.read_csv(DATA_PATH/'val.csv', delimiter=',')

all_data = pd.concat([train_df, test_df, val_df])

class_names = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 
              'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# For performance reasons, only going to use 6,000 Tweets.
batch_tweets = all_data[:6000]
#batch_labels = all_data[class_names][:6000]

In [25]:
batch_tweets['Tweet'].head()

0    “Worry is a down payment on a problem you may ...
1    Whatever you decide to do make sure it makes y...
2    @Max_Kellerman  it also helps that the majorit...
3    Accept the challenges so that you can literall...
4    My roommate: it's okay that we can't spell bec...
Name: Tweet, dtype: object

In [5]:
len(batch_tweets)

6000

# Load Pre-Trained BERT Model

In [6]:
model_class, tokenizer_class, pretrained_weights = (
    ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (dropout): Dropout(p=0.1, inplace=False)
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_fea

# Preparing the Dataset

## Tokenization

In [0]:
tokenized = batch_tweets['Tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

## Padding

In [0]:
max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)

padded = np.array([i + [0] * (max_len-len(i)) for i in tokenized.values])

In [9]:
np.array(padded).shape

(6000, 71)

## Masking

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(6000, 71)

# Run Model

In [0]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_sates = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_sates[0][:,0,:].numpy()

In [13]:
features

array([[ 0.04521827, -0.19778816, -0.14963762, ..., -0.29783145,
         0.00215782,  0.5573782 ],
       [-0.07956323, -0.08615977,  0.25079146, ..., -0.24437697,
         0.12766258,  0.29017454],
       [-0.06133094, -0.0864303 , -0.10019764, ..., -0.04994415,
         0.2807403 ,  0.39241007],
       ...,
       [-0.24905805,  0.00395511, -0.02915181, ..., -0.0561239 ,
         0.20778862,  0.28375342],
       [-0.02059313, -0.11244061, -0.06061284, ..., -0.0480255 ,
         0.4726586 ,  0.23896316],
       [-0.19081639, -0.24672167, -0.07189524, ...,  0.01926764,
         0.52206874,  0.2643463 ]], dtype=float32)

In [14]:
len(features)

6000

In [31]:
labels = batch_tweets[class_names]
labels

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0,1,0,0,0,0,1,0,0,0,1
1,0,0,0,0,1,1,1,0,0,0,0
2,1,0,1,0,1,0,1,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5995,0,0,0,1,0,0,0,0,0,0,0
5996,1,0,1,0,0,0,0,0,0,0,0
5997,1,0,1,0,0,0,0,0,0,0,0
5998,1,0,1,0,0,0,0,0,0,0,0


In [0]:
train_features = features[:4800]
train_labels = labels[:4800]

test_features = features[4800:]
test_labels = labels[4800:]

In [33]:
test_labels

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
4800,1,0,1,1,0,0,0,0,0,0,0
4801,1,0,1,0,1,0,0,0,0,0,0
4802,0,0,0,1,0,0,1,0,0,0,0
4803,0,0,0,0,0,0,0,0,1,0,0
4804,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5995,0,0,0,1,0,0,0,0,0,0,0
5996,1,0,1,0,0,0,0,0,0,0,0
5997,1,0,1,0,0,0,0,0,0,0,0
5998,1,0,1,0,0,0,0,0,0,0,0


In [0]:
from sklearn.metrics import accuracy_score

In [35]:
clf = LogisticRegression()

scores = []

for class_name in class_names:
  print('\n... Processing {}'.format(class_name))

  train_target = train_labels[class_name]
  test_target = test_labels[class_name]

  cv_score = np.mean(cross_val_score(clf, train_features, train_target, cv=3, scoring='roc_auc'))
  scores.append(cv_score)
  
  print('CV score for class {} is {}'.format(class_name, cv_score))

  clf.fit(train_features, train_target)

  # Compute training accuracy
  y_pred_train = clf.predict(train_features)
  print('Training accuracy is {}'.format(accuracy_score(train_target, y_pred_train)))

  # Compute testing accuracy 
  y_pred_test = clf.predict(test_features)
  print('Testing accuracy is {}'.format(accuracy_score(test_target, y_pred_test)))

print('\nTotal CV score is {}'.format(np.mean(scores)))


... Processing anger
CV score for class anger is 0.8741448498637294
Training accuracy is 0.8416666666666667
Testing accuracy is 0.805

... Processing anticipation
CV score for class anticipation is 0.6888349249458376
Training accuracy is 0.8789583333333333
Testing accuracy is 0.8525

... Processing disgust
CV score for class disgust is 0.8313324725801828
Training accuracy is 0.8085416666666667
Testing accuracy is 0.77

... Processing fear
CV score for class fear is 0.8407506186760134
Training accuracy is 0.8983333333333333
Testing accuracy is 0.8583333333333333

... Processing joy
CV score for class joy is 0.8741551768527754
Training accuracy is 0.84875
Testing accuracy is 0.84

... Processing love
CV score for class love is 0.8704612738470631
Training accuracy is 0.9310416666666667
Testing accuracy is 0.905

... Processing optimism
CV score for class optimism is 0.8365824692996252
Training accuracy is 0.8441666666666666
Testing accuracy is 0.8066666666666666

... Processing pessimism